In this script, we subsampled 2100 Toronto geolocations from a Toronto geolocation dataset.

With the 2100 geolocations, we were able to fetch 2034 Toronto images.

Those images would be used to deploy our final model for the predictions of Perceived Toronto Street View. 

The original data were fetched from: https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#f71a13c4-fb51-6116-57b7-1f51a8190585 There was no zoning / subdistrict c was not avaliable. 

In [2]:
import geopandas as gp
import pandas as pd
from shapely.geometry import Point, Polygon

In [2]:
data = gp.read_file("ADDRESS_POINT_WGS84.shp")
print(data.shape)
data.head(10)

(525545, 23)


GEO_ID     LINK MAINT_STAG ADDRESS            LFNAME  LONUM LONUMSUF  \
0   1015200   914764    REGULAR      22  Lloyd George Ave     22     None   
1   1015189   914764    REGULAR       3  Lloyd George Ave      3     None   
2  20140046   914764    REGULAR      7A  Lloyd George Ave      7        A   
3   1001594   914677    REGULAR      58          Foch Ave     58     None   
4   1001593   914677    REGULAR      54          Foch Ave     54     None   
5   1001595   914677    REGULAR      60          Foch Ave     60     None   
6   1001597   914677    REGULAR      62          Foch Ave     62     None   
7   1001599   914677    REGULAR      64          Foch Ave     64     None   
8   7275227  7275212    REGULAR      46      Jellicoe Ave     46     None   
9   7275226  7275212    REGULAR      44      Jellicoe Ave     44     None   

   HINUM HINUMSUF ARC_SIDE  ...  CLASS  NAME           X            Y  \
0      0     None        L  ...   Land  None  301211.152  4827985.051   
1      0     None        R  ...   Land  None  301272.174  4827946.759   
2      0     None        R  ...   Land  None  301265.553  4827966.490   
3      0     None        L  ...   Land  None  301024.624  4828252.320   
4      0     None        L  ...   Land  None  301037.208  4828229.086   
5      0     None        L  ...   Land  None  301023.299  4828264.664   
6      0     None        L  ...   Land  None  301017.022  4828274.464   
7      0     None        L  ...   Land  None  301010.239  4828286.590   
8      0     None        R  ...   Land  None  301072.405  4828239.453   
9      0     None        R  ...   Land  None  301083.102  4828246.611   

   LONGITUDE   LATITUDE   OBJECTID   MUN_NAME            WARD_NAME  geometry  
0 -79.544251  43.593789  4104309.0  Etobicoke  Etobicoke-Lakeshore      None  
1 -79.543495  43.593445  4104306.0  Etobicoke  Etobicoke-Lakeshore      None  
2 -79.543577  43.593622  4104315.0  Etobicoke  Etobicoke-Lakeshore      None  
3 -79.546567  43.596203  2842769.0  Etobicoke  Etobicoke-Lakeshore      None  
4 -79.546411  43.595994  2005187.0  Etobicoke  Etobicoke-Lakeshore      None  
5 -79.546583  43.596314  1977825.0  Etobicoke  Etobicoke-Lakeshore      None  
6 -79.546661  43.596402  1570005.0  Etobicoke  Etobicoke-Lakeshore      None  
7 -79.546745  43.596511  1585038.0  Etobicoke  Etobicoke-Lakeshore      None  
8 -79.545975  43.596087  1558852.0  Etobicoke  Etobicoke-Lakeshore      None  
9 -79.545842  43.596152  1573857.0  Etobicoke  Etobicoke-Lakeshore      None  

[10 rows x 23 columns]

In [3]:
list(data)

['GEO_ID',
 'LINK',
 'MAINT_STAG',
 'ADDRESS',
 'LFNAME',
 'LONUM',
 'LONUMSUF',
 'HINUM',
 'HINUMSUF',
 'ARC_SIDE',
 'DISTANCE',
 'FCODE',
 'FCODE_DES',
 'CLASS',
 'NAME',
 'X',
 'Y',
 'LONGITUDE',
 'LATITUDE',
 'OBJECTID',
 'MUN_NAME',
 'WARD_NAME',
 'geometry']

In [5]:
data['geometry'].isna().sum()

525545

#### Randaomly choose 2010 locations, as we plan to use 2000 images, and some location's street view could not be fetched.

In [29]:
Toronto2100_sample = data[['LONGITUDE','LATITUDE', 'MUN_NAME', 'WARD_NAME']].sample(n=2100)

#### Original data's count by Ward_Name

In [25]:
data.groupby(['MUN_NAME', 'WARD_NAME']).count()

GEO_ID   LINK  MAINT_STAG  ADDRESS  \
MUN_NAME       WARD_NAME                                                      
East York      Beaches-East York          10747  10747       10747    10747   
               Don Valley East                1      1           1        1   
               Don Valley West             5348   5348        5348     5348   
               Toronto-Danforth            7702   7702        7702     7702   
               University-Rosedale          221    221         221      221   
Etobicoke      Etobicoke Centre           25665  25665       25665    25665   
               Etobicoke North            18752  18752       18752    18752   
               Etobicoke-Lakeshore        29104  29104       29104    29104   
North York     Beaches-East York            101    101         101      101   
               Don Valley East            10722  10722       10722    10722   
               Don Valley North           18513  18513       18513    18513   
               Don Valley West             8750   8750        8750     8750   
               Eglinton-Lawrence          13236  13236       13236    13236   
               Humber River-Black Creek   17826  17826       17826    17826   
               Willowdale                 18537  18537       18537    18537   
               York Centre                18661  18661       18661    18661   
               York South-Weston           7876   7876        7876     7876   
Scarborough    Beaches-East York             72     72          72       72   
               Scarborough Centre         19895  19895       19895    19895   
               Scarborough North          19890  19890       19890    19890   
               Scarborough Southwest      24850  24850       24850    24850   
               Scarborough-Agincourt      16652  16652       16652    16652   
               Scarborough-Guildwood      16785  16785       16785    16785   
               Scarborough-Rouge Park     25762  25762       25762    25762   
York           Davenport                   5031   5031        5031     5031   
               Eglinton-Lawrence           2903   2903        2903     2903   
               Parkdale-High Park          3664   3664        3664     3664   
               Toronto-St. Paul's          6787   6787        6787     6787   
               York South-Weston          14091  14091       14091    14091   
former Toronto Beaches-East York          17873  17873       17873    17873   
               Davenport                  23697  23697       23697    23697   
               Don Valley West             7803   7803        7803     7803   
               Eglinton-Lawrence           9898   9898        9898     9898   
               Parkdale-High Park         19304  19304       19304    19304   
               Spadina-Fort York          10361  10361       10361    10361   
               Toronto Centre              9254   9254        9254     9254   
               Toronto-Danforth           23216  23216       23216    23216   
               Toronto-St. Paul's         12876  12876       12876    12876   
               University-Rosedale        21762  21762       21762    21762   
               York South-Weston           1357   1357        1357     1357   

                                         LFNAME  LONUM  LONUMSUF  HINUM  \
MUN_NAME       WARD_NAME                                                  
East York      Beaches-East York          10747  10747       264  10747   
               Don Valley East                1      1         0      1   
               Don Valley West             5348   5348       193   5348   
               Toronto-Danforth            7702   7702       210   7702   
               University-Rosedale          221    221         7    221   
Etobicoke      Etobicoke Centre           25665  25665       355  25665   
               Etobicoke North            18752  18752       394  18752   
               Etobicoke-Lakeshore        29104

#### Sampled data's count by Ward_Name

In [30]:
Toronto2100_sample.groupby(['MUN_NAME', 'WARD_NAME']).count()

LONGITUDE  LATITUDE
MUN_NAME       WARD_NAME                                    
East York      Beaches-East York                45        45
               Don Valley West                  18        18
               Toronto-Danforth                 30        30
               University-Rosedale               1         1
Etobicoke      Etobicoke Centre                 89        89
               Etobicoke North                  59        59
               Etobicoke-Lakeshore             122       122
North York     Don Valley East                  39        39
               Don Valley North                 73        73
               Don Valley West                  38        38
               Eglinton-Lawrence                51        51
               Humber River-Black Creek         86        86
               Willowdale                       80        80
               York Centre                      95        95
               York South-Weston                30        30
Scarborough    Scarborough Centre               84        84
               Scarborough North                78        78
               Scarborough Southwest            95        95
               Scarborough-Agincourt            67        67
               Scarborough-Guildwood            76        76
               Scarborough-Rouge Park           96        96
York           Davenport                        18        18
               Eglinton-Lawrence                 9         9
               Parkdale-High Park               13        13
               Toronto-St. Paul's               23        23
               York South-Weston                54        54
former Toronto Beaches-East York                70        70
               Davenport                        82        82
               Don Valley West                  25        25
               Eglinton-Lawrence                32        32
               Parkdale-High Park              103       103
               Spadina-Fort York                31        31
               Toronto Centre                   36        36
               Toronto-Danforth                 95        95
               Toronto-St. Paul's               58        58
               University-Rosedale              92        92
               York South-Weston                 7         7

#### Sampled data's count by MUN_NAME

In [31]:
Toronto2100_sample.groupby(['MUN_NAME']).count()

LONGITUDE  LATITUDE  WARD_NAME
MUN_NAME                                      
East York              94        94         94
Etobicoke             270       270        270
North York            492       492        492
Scarborough           496       496        496
York                  117       117        117
former Toronto        631       631        631

In [32]:
Toronto2100_sample.to_csv("~/Desktop/ML1030/Toronto2100_sample.csv")

#### We were able to fetch 2034 images from the 2100 locations using Google Street View API.

In [6]:
pd.read_json("/Users/zhanglingling/Desktop/ML1030/toronto_metadata.json").head()

_file copyright       date  \
0  gsv_0.jpg  © Google 2018-07-01   
1  gsv_1.jpg  © Google 2018-08-01   
2  gsv_2.jpg  © Google 2018-08-01   
3  gsv_3.jpg  © Google 2018-08-01   
4  gsv_4.jpg  © Google 2018-08-01   

                                            location                 pano_id  \
0  {'lat': 43.6814075014274, 'lng': -79.444445154...  aLRQFLoboL6papa2qE1Veg   
1  {'lat': 43.7488349459349, 'lng': -79.492156396...  hwo_vjTqlVcqF-my9fjVhw   
2  {'lat': 43.75852320755994, 'lng': -79.18330303...  1yc7mFQX1OGCMh2n9OvtRQ   
3  {'lat': 43.64732825618856, 'lng': -79.40845388...  hHP4wWeB0Ak1eqRV6ncfhg   
4            {'lat': 43.7604919, 'lng': -79.1795913}  bw_AvWlyXIOe2rV39MLNqQ   

  status  
0     OK  
1     OK  
2     OK  
3     OK  
4     OK

In [7]:
pd.read_json("/Users/zhanglingling/Desktop/ML1030/toronto_metadata.json").groupby('status').size()

status
OK              2034
ZERO_RESULTS      66
dtype: int64